<a href="https://colab.research.google.com/github/pedrolustosab/parametros_powerquery/blob/main/pdr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [2]:
# Informações de conexão ao banco de dados
db_user = userdata.get('USER')
db_password = userdata.get('PASSWORD')
db_host = userdata.get('SEVER')
db_name = userdata.get('DB')

# Criar a string de conexão
db_url = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}/{db_name}'

# Criar a engine SQLAlchemy
engine = create_engine(db_url)

# Testar a conexão
try:
    connection = engine.connect()
    print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro de conexão: {e}")

Conexão bem-sucedida!


In [3]:
# Definindo os parâmetros
num_records = 1000000
num_days = 3 * 365 + 174  # 3 anos
start_date = datetime.now() - timedelta(days=num_days)

# Gerando as datas de vendas
date_range = pd.date_range(start=start_date, periods=num_days).to_pydatetime().tolist()
dates = np.random.choice(date_range, num_records)

# Gerando IDs de Lojas seguindo a proporção 50-30-20
stores = np.random.choice([1, 2, 3], num_records, p=[0.5, 0.3, 0.2])

# Gerando IDs de Produtos aleatórios (supondo que existem 100 produtos diferentes)
products = np.random.randint(1, 20, num_records)

# Gerando os valores de venda aleatórios
sales_values = np.round(np.random.uniform(3, 60, num_records), 2)

# Criando o DataFrame
df_sales = pd.DataFrame({
    'DT_Venda': dates,
    'ID_Loja': stores,
    'ID_Produto': products,
    'VL_Venda': sales_values
})

# Convertendo a coluna de datas para o formato date
df_sales['DT_Venda'] = pd.to_datetime(df_sales['DT_Venda']).dt.date



In [4]:
df_sales

,DT_Venda,ID_Loja,ID_Produto,VL_Venda
0,2024-04-20,2,10,45.27
1,2023-12-01,1,14,35.48
2,2023-03-11,1,1,56.67
3,2023-10-24,2,6,22.79
4,2021-03-20,1,17,44.57
...,...,...,...,...
999995,2024-06-15,1,18,22.37
999996,2022-10-09,3,11,58.51
999997,2024-04-11,2,15,47.79
999998,2021-03-10,1,13,10.85


In [5]:
df_sales.describe()

,ID_Loja,ID_Produto,VL_Venda
count,1000000.000000,1000000.000000,1000000.000000
mean,1.700816,10.001104,31.506733
std,0.781516,5.477653,16.448454
min,1.000000,1.000000,3.000000
25%,1.000000,5.000000,17.260000
50%,2.000000,10.000000,31.520000
75%,2.000000,15.000000,45.740000
max,3.000000,19.000000,60.000000


In [6]:
# Cria um dicionário de tradução para os nomes dos meses e abreviações
traducao_meses = {
    1: 'Janeiro',
    2: 'Fevereiro',
    3: 'Março',
    4: 'Abril',
    5: 'Maio',
    6: 'Junho',
    7: 'Julho',
    8: 'Agosto',
    9: 'Setembro',
    10: 'Outubro',
    11: 'Novembro',
    12: 'Dezembro'
}

traducao_abreviacoes = {
    1: 'Jan',
    2: 'Fev',
    3: 'Mar',
    4: 'Abr',
    5: 'Mai',
    6: 'Jun',
    7: 'Jul',
    8: 'Ago',
    9: 'Set',
    10: 'Out',
    11: 'Nov',
    12: 'Dez'
}

In [7]:
# Obtém a data máxima e mínima do dataframe df
data_max = df_sales['DT_Venda'].max()
data_min = df_sales['DT_Venda'].min()

#Cria um dataframe com datas entre a maxima e minima
dcalendario = pd.DataFrame()
dcalendario['date'] = pd.date_range(data_min, data_max)

# Cria as outras colunas conforme solicitado
dcalendario['ano'] = dcalendario['date'].dt.year
dcalendario['mesano'] = dcalendario['date'].dt.strftime('%Y%m').astype(int)  # Converte para inteiro
dcalendario['nmes'] = dcalendario['date'].dt.month
# Traduz os nomes dos meses e suas abreviações
dcalendario['nome_mes'] = dcalendario['date'].dt.month.map(traducao_meses)
dcalendario['mesabrev'] = dcalendario['date'].dt.month.map(traducao_abreviacoes)
dcalendario['day'] = dcalendario['date'].dt.day

# Mostra as últimas 3 linhas do novo dataframe
print(dcalendario.tail(3))

           date   ano  mesano  nmes nome_mes mesabrev  day
1266 2024-07-20  2024  202407     7    Julho      Jul   20
1267 2024-07-21  2024  202407     7    Julho      Jul   21
1268 2024-07-22  2024  202407     7    Julho      Jul   22


In [8]:
# Loja
loja = {
    'nome_loja': ['Matriz', 'Aldeota', 'Meireles'],
    'id_loja': [1, 2, 3]
}

# Cria um dataframe de tópicos
df_loja = pd.DataFrame(loja)
df_loja

,nome_loja,id_loja
0,Matriz,1
1,Aldeota,2
2,Meireles,3


In [9]:
# Escrever os dados na tabela
df_sales.to_sql('fVendas', engine, schema='SALES', index=False, if_exists='replace')
dcalendario.to_sql('dcalendario', engine, schema='SALES', index=False, if_exists='replace')
df_loja.to_sql('dloja', engine, schema='SALES', index=False, if_exists='replace')

3

In [10]:
 connection.close()